## 9.파이썬스러운 객체
### 9.1 객체 표현
<li><b>repr( )</b></li>
객체를 개발자가 보고자 하는 형태로 표현한 문자열로 반환한다. __repr__( ) 특별 메서드를 구현해야 한다.
<li><b>str( )</b></li>
객체를 사용자가 보고자 하는 형태로 표현한 문자열로 반환한다. __str__( ) 특별 메서드를 구현해야 한다.
<li><b>bytes( )</b></li>
__bytes__( )는 __str__()과 비슷하지만 bytes() 메서드에 의해 호출되어 객체를 바이트 시퀀스로 표현한다.
<li><b>format( )</b></li>
__format__( )은 내장 함수 format()과 str.format() 메서드 둘다 사용하며, 특별 포맷 코드를 이용해서 객체를 표현하는 문자열을 반환한다.

 <b>\_\_repr\_\_( )</b>, <b>\_\_str\_\_( )</b>, <b>\_\_format\_\_( )</b> 메서드는 반드시 유니코드 문자열(str형)을 반환해야 한다. <b>\_\_bytes\_\_( )</b>만 바이트 시퀀스를 반환한다.

### 9.2 백터 클래스의 부활
객체 표현과 관련된 여러 메서드의 예를 살펴보기 위해 Vector2d 클래스를 사용한다.

In [1]:
""" [예제 9-2] vector2d_v0.py : 지금까지의 메서드는 모두 특별 메서드이다. """

from array import array
import math

class Vector2d():
#     __slots__ = ('__x', '__y') # 이 속성들이 클래스 객체가 가지는 속성임을 인터프리터에 알려줌
    typecode = 'd'  
    
    def __init__(self, x, y):
        self.__x = float(x) # 정확히 두 개의 언더바로 시작해서(뒷부분의 언더바는 없거나 하나만 사용한다) 속성을 비공개로 만든다.
        self.__y = float(y)
        
    @property # 프로퍼티의 getter 메서드를 나타낸다.
    def x(self): # 자신이 노출시키는 공개 속성명을 따라 게터 메서드의 이름을 지정한다.
        return self.__x 
    
    @property 
    def y(self):
        return self.__y
    
    def __iter__(self): # __iter__()를 구현하면 Vector2d를 반복할 수 있게된다. 때문에 x, y = my_vector 문장으로 언패킹 할 수 있다. 
        return (i for i in (self.x, self.y)) 
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self) # __iter__() 구현으로 반복이 가능해져서 self.x, self.y를 전달한다.
    
    def __str__(self):
        return str(tuple(self)) # 반복형 Vector2d에서 튜플을 만들어 순서쌍으로 출력
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other) # 모든 속성을 비교하기 위해 피연산로부터 튜플을 생성한다. 
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    @classmethod                                     # 클래스 메서드에는 @classmethod 데커레이터가 붙는다. (9.4에서 소개함)
    def frombytes(cls, octets):                      # self 매개변수가 없다. 대신 클래스 자신이 cls 매개변수로 전달된다.
        typecode = chr(otctets[0])                   # 첫 번째 바이트에서 typecode를 읽는다.
        memv = memoryview(octets[1:]).cast(typecode) # 이진 시퀀스로부터 memoryview를 생성하고 typecode를 이용해서 형을 변환한다.
        return cls(*memv)                            # cast()가 반환한 memoryview를 언패킹해서 생성자에게 필요한 인수로 전달한다.

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle()) # (크기, 각)으로 극좌표 튜플을 만든다.
            outer_fmt = '<{}, {}>'
        else:
            coords = (abs(self), self.angle())
            outer_fmt = '({}, {})'

        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def angle(self):
        return math.atan2(self.y, self.x)

In [2]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y) # Vector2d 요소들은 게터 메서드를 호출할 필요없이 직접 속성에 접근할 수 있다.
    
x, y = v1
print(x, y)      # Vector2d를 변수들의 튜플에 언패킹 할 수 있다.
v1               # Vector2d의 repr()은 객체를 생성하는 소스코드와 같은 형태를 출력한다.

In [3]:
v1_clone = eval(repr(v1)) # The eval() function evaluates the specified expression, if the expression is a legal Python statement, it will be executed.
v1_clone == v1            # 즉 repr(v1)의 결과는 'Vector2d(3.0, 4.0)' 이며, 이 string을 eval() 함수를 통해 실행했기 때문에 v1_clone도 v1과 같은 값을 갖게 된다.

In [4]:
print(v1) # __str__()의 실행결과가 출력된다.

In [5]:
octets = bytes(v1) # __bytes__()의 실행결과가 출력된다.
octets

In [6]:
abs(v1) # Return the Euclidean distance, sqrt(x*x + y*y).

In [7]:
bool(v1), bool(Vector2d(0, 0)) # Vector2d의 크기가 0이면 False 나머지는 1이다.

### 9.3 대안 생성자
Vector2d를 bytes로 변환하는 메서드가 있으니, bytes를 Vector2d로 변환하는 메서드도 있어야 할 것이다. 영감을 얻기 위해 표준 라이브러리를 살펴보면, frombytes()라는 클래스 메서드를 가진 array.array가 우리 상황에 딱 맞는 것 같다. 이를 이용해서 Vectro2d의 클래스 메서드를 추가해보자.
```
""" [예제 9-3] """

@classmethod                                     # 클래스 메서드에는 @classmethod 데커레이터가 붙는다. (9.4에서 소개함)
def frombytes(cls, octets):                      # self 매개변수가 없다. 대신 클래스 자신이 cls 매개변수로 전달된다.
    typecode = chr(otctets[0])                   # 첫 번째 바이트에서 typecode를 읽는다.
    memv = memoryview(octets[1:]).cast(typecode) # 이진 시퀀스로부터 memoryview를 생성하고 typecode를 이용해서 형을 변환한다.
    return cls(*memv)                            # cast()가 반환한 memoryview를 언패킹해서 생성자에게 필요한 인수로 전달한다.
```

### 9.4 @classmethod와 @staticmethod

먼저 @classmethod부터 살펴보면, [예제 9-3]에서 보듯이 @classmethod 데커레이터는 객체가 아닌 클래스에 연산을 수행하는 메서드를 정의한다는 것을 알 수 있다. @classmethod는 메서드가 호출되는 방식을 변경해서 클래스 자체를 첫 번째 인수로 받게 만들며, [예제 9-3]에서 본 frombytes()와 같은 대안 생성자를 구현하기 위해 주로 사용된다. frombytes()메서드의 마지막 문장에서 cls(*memv)는 객체를 생성하기 위해 cls인수를 이용해서 실제로 클래스의 생성자를 호출한다. 관습적으로 cls를 클래스 메서드의 첫 번째 매개변수명으로 사용하지만, 파이썬은 특정 매개변수명을 요구하지 않는다.

반대로 @staticmethod 데커레이터는 메서드가 특별한 첫 번째 인수를 받지 않도록 메서드를 변경한다. 본질적으로 정적 메서드는 모듈 대신 클래스 본체 안에 정의된 평범한 함수일 뿐이다. [예제 9-4]는 @classmethod와 @staticmethod 데커레이터의 동작을 비교해서 보여준다.

In [8]:
""" [예제9-4] @classmethod와 @staticmethod 동작 비교 """

class Demo:
    
    @classmethod
    def klassmeth(*args):
        return args # 모든 위치의 인수를 보여준다.
    
    @staticmethod
    def statmeth(*args):
        return args # 마찬가지로 모든 위치의 인수를 보여준다.
                    # ※ 필자는 클래스와 함께 작동하지 않는 함수를 정의하려면 단지 모듈에 정의하면 된다고 생각하기 때문에 활용성에 대해서는 부정적이다.
                    # ※ 필자와 다른 생각의 참고자료 
                    # The Definitive Guide on Tho to Use Static, Class or Abstract Methods in Python written by Julien Danjou            

In [9]:
Demo.klassmeth()

(__main__.Demo,)

In [10]:
Demo.klassmeth('spam') # 호출 방법에 무관하게 Demo.klassmeth()는 Demo 클래스를 첫 번째 인수로 받는다.

(__main__.Demo, 'spam')

In [11]:
Demo.statmeth()

()

In [12]:
Demo.statmeth('spam') # 는 단지 평범한 함수처럼 동작할 뿐이다.

('spam',)

### 9.5 포맷된 출력
format() 내장 함수와 str.format() 메서드는 실제 포맷 작업을 \_\_format\_\_(format_spec) 메서드에 위임한다. format_spec은 포맷 명시자<sup>format specifier</sup>로서, 다음 두 가지 방법 중 하나를 통해 지정한다.
<li> format(my_obj, format_spec)의 두 번째 인수 </li>
<li> str.format()에 사용된 포맷 문자열 안에 {}로 구분한 대체 필드 안에서 콜론 뒤의 문자열 </li>

예를 들어 다음과 같이 사용한다.

In [13]:
br1 = 1/2.43
print(br1)
print(format(br1, '0.4f'))
print('1 BRL = {rate:0.2f} USD'.format(rate=br1))

0.4115226337448559
0.4115
1 BRL = 0.41 USD


몇몇 내장 자료형은 자신만의 고유한 표현 코드를 가지고 있다. 예를 들어 int형의 경우 이진수를 나타내는 'b', 16진수를 나타내는 'x' 코드를 지원하며, float형의 경우 고정소수점을 나타내는 ''f, 백분율을 나타내는 '%' 코드를 지원한다. 

In [14]:
print(format(42, 'b'))
print(format(2/3, '.1%'))

101010
66.7%


각 클래스가 format_spec 인수를 자신이 원하는 대로 확장할 수 있다. 예를 들어 datetime 모듈의 클래스들은 자신의 \_\_format\_\_() 메서드에서 strftime() 함수와 동일한 포맷 코드를 사용한다.

In [15]:
from datetime import datetime
now = datetime.now()
print(format(now, '%H:%M:%S'))
print("It's now {:%I %M %p}".format(now))

12:17:15
It's now 12 17 PM


클래스에서 \_\_format\_\_() 메서드를 정의하지 않으면, object에서 상속받은 메서드가 str(my_object)를 반환한다.

In [16]:
v1 = Vector2d(3, 4)
format(v1)

'(5.0, 0.9272952180016122)'

그러나 이때 포맷 명시자를 사용하면 object. \_\_format\_\_()은 TypeError를 발생한다. 이 문제를 해결하려면 \_\_format\_\_() 메서드를 구현해야 한다.
```
    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({}. {})'.format(*components)
```

In [17]:
format(v1, '.3f')

'(5.000, 0.927)'

In [18]:
v1 = Vector2d(3,4)
print(format(v1))

print(format(v1, '.2f'))
print(format(v1, '.3e'))

(5.0, 0.9272952180016122)
(5.00, 0.93)
(5.000e+00, 9.273e-01)


이제 Vector2d의 포맷 코드를 추가해보자. 포맷 명시자가 'p'로 끝나면 백터를 극좌표 <Γ, θ>로 표현한다. Γ은 백터의 크기, θ는 라디안으로 표현된 각을 나타낸다.
```
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle()) # (크기, 각)으로 극좌표 튜플을 만든다.
            outer_fmt = '<{}, {}>'
        else:
            coords = (abs(self), self.angle())
            outer_fmt = '({}, {})'

        components = (format(c, fmt_spec) for c in self)
        return outer_fmt.format(*components)
    
    def angle(self):
        return math.atan2(self.y, self.x)
```

In [19]:
print(format(Vector2d(1, 1), 'p'))
print(format(Vector2d(1, 1), '.3ep'))
print(format(Vector2d(1, 1), '.5fp'))

<1.4142135623730951, 0.7853981633974483>
<1.414e+00, 7.854e-01>
<1.41421, 0.78540>


### 9.6 해시 가능한 Vector2d
지금까지 정의한 Vector2d는 해시할 수 없다. 그러므로 집합 안의 항목으로 사용할 수 없다. Vector2d를 해시 가능하게 만들려면 \_\_hash\_\_( ), \_\_eq\_\_( )<sup>이미 구현함</sup> 메서드를 구현해야 한다. 그리고 3.1절의 '해시 가능하다는 말의 의미는?' 글상자에서 설명한 것처럼 불변형으로 만들어야 한다. v1.x=7과 같이 속성을 변경하는 코드를 입력하면, 현재 구현된 Vector2d가 불변형이 아니므로 에러가 발생한다.

In [20]:
v1 = Vector2d(3, 4)
hash(v1)

7

In [21]:
set([v1])

{Vector2d(3.0, 4.0)}

In [26]:
print(v1.x, v1.y)
v1.x = 7

3.0 4.0


AttributeError: can't set attribute

In [27]:
print(v1.x) # 책에서는 안된다고 하는데 여기는 됨

3.0


Vector2d를 불변형으로 만들었으니, 이제 \_\_hash\_\_( ) 메서드를 구형할 차례다. \_\_hash\_\_( ) 메서드는 int 형을 반환해야 한다. 그리고 동일하다고 판단되는 객체는 일링한 해시값을 가져야 하므로, \_\_eq\_\_( ) 메서드가 사용하는 객체의 속성을 이용해서 해시를 계산하는 것이 이상적이다. \_\_hash\_\_( ) 메서드 문서 (https://docs.python.org/3/reference/datamodel.html) 에서는 요소의 해시에 비트 단위 XOR(베타적 논리합) 연산자(^)를 사용하는 것을 권장하므로 여기서도 그 방법을 따른다.

```
def __hash__(self):
    return hash(self.x) ^ hash(self.y)
```

해시 가능형을 만들기 위해 반드시 프로퍼티를 구현하거나 객체 속성을 보호할 필요는 없다. 단지 \_\_hash()\_\_ 와 \_\_eq()\_\_ 메서드를 제대로 구현하면 된다. 그러나 객체의 해시값이 변하면 안되므로 읽기 전용 프로퍼티를 설명하기 좋은 기회이긴 하다.

In [28]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
print(hash(v1), hash(v2))
print(set([v1, v2]))

7 384307168202284039
{Vector2d(3.1, 4.2), Vector2d(3.0, 4.0)}


적절한 스칼라 값을 가진 자료형을 만들 때는 경우에 따라 자료형을 강제 변환하기 위해 사용되는 int( )와 float( )가 호출하는 \_\_int\_\_( )와 \_\_float\_\_( ) 메서드를 구현하는 것도 좋다. 내장된 complex( ) 생성자를 지원하기 위한 \_\_complex\_\_( ) 메서드도 있다.

### 9.7 파이썬에서 비공개 속성과 보호된 속성
파이썬에서 private 수정자가 있는 자바와 달리 비공개 변수를 생성할 수 있는 방법은 없지만 서브클래스에서 '비공개' 성격의 속성을 실수로 변경하지 못하게 하는 간단한 메커니즘은 있다. 

이런 시나리오를 생각해보자. 클래스 외부에 노출시키지 않고 내부적으로 개의 상태를 나타내는 mood 객체 속성을 사용하는 Dog라는 클래스가 있다. 우리는 Dog를 상속해서 Beagle이라는 클래스를 정의해야 한다. 이때 Dog에 mood라는 속성이 있는지 모르고 Beagle에서 mood라는 속성을 정의하느 경우 이름 충돌을 일으키게 된다. 

이런 상황을 예방하기 위해 속성명을 \_\_mood 처럼 두개의 언더바로 시작하고 언더바 없이 또는 하나의 언더바로 끝나도록 정의하면, 파이썬은 언더바와 클래스명을 변수명 앞에 붙여 객체의 \_\_dict\_\_에 저장한다. 따라서 Dog클래스의 경우 \_\_mood는 \_Dog\_\_mood가 되고 beagle 클래스의 경우 \_Beagle\_\_mood가 된다. 이러한 파이썬 언어 기능을 이름 장식<sup>name mangling</sup> 이라고 한다.

In [29]:
""" [예제 9-10] 클래스명을 앞에 붙여 비공개 속성명 장식하기 """
v1 = Vector2d(3, 4)
print('{!r}'.format(v1.__dict__))
print(v1._Vector2d__x)

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}
3.0


속성명 앞에 언더바 하나를 붙이러더라도 파이썬 인터프리터가 별도로 특별히 처리하는 것은 없지만, 클래스 외부에서 그런 속성에 접근하지 않는 것은 프로그래머 사이에 일종의 금기처럼 자리잡혀 있다. 언더바 하나를 앞에 붙여서 표시한 객체의 프라이버시를 존중하는 것은 모든 글자를 대분자로 사용하는 상수를 존중하는 것처럼 간단한 일이다. 

정리하면, Vector2d 요소는 비공개속성이며, Vector2d 객체는 불변형이다. 

### 9.8 \_\_slots\_\_ 클래스 속성으로 공간 절약하기
기본적으로 파이썬은 객체 속성을 각 객체안의 \_\_dict\_\_ 이라는 딕셔너리형 속성에 저장한다. 딕셔너리는 빠른 접근 속도를 제공하기 위해 내부에 해시 테이블을 유지하므로 메모리 사용량이 많다. \_\_slots\_\_ 속성은 파이썬 인터프리터가 객체 속성을 딕셔너리 대신 튜플에 저장하게 만든다.

In [30]:
""" [예제 9-11] Vector2d 클래스에 __slot__ 속성 추가 """

class Vectory2d:
    __slots__ = ('__x', '__y') # 이 속성들이 클래스 객체가 가지는 속성임을 인터프리터에 알려줌
    typecode = 'd'  

\_\_slots\_\_를 사용할 때는 주의할 점이 있으며 단지 클래스의 사용자가 할당할 수 있는 속성을 제한하기 위해 사용해서는 안 된다. \_\_slots\_\_는 고정된 스키마의 아주 큰 데이터베이스와 같은 테이블 형태의 데이터를 사용할 때 유용하다. 그러나 이런 형태의 데이터를 자주 처리해야 한다면 numpy나 pandas 데이터 분석 라이브러리를 알아보는 것이 좋다. 

#### 9.8.1 __slots__를 사용할 때 주의할 점
<p>
<li> 인터프리터는 상속된 __slots__ 속성을 무시하므로 각 클래스마다 __slots__ 속성을 다시 정의해야 한다. </li>
<li> __dict__를 __slot__에 추가하지 않는 한 객체는 __slot__에 나열된 속성만 가질 수 있다.(그러나 __dict__를 __slot__에 추가하면 메모리 절감 효과가 반감될 수 있다.)  </li>
<li> __weakref__를 __slots__에 추가하지 않으면 객체가 약한 참조의 대상이 될 수 없다.</li></p>

### 9.9 클래스 속성 오버라이드
클래스 속성을 객체 속성의 기본값으로 사용하는 것은 파이썬의 독특한 특징이다. Vector2d 클래스에는 typecode라는 클래스 속성이 있다. 이 속성은 \_\_bytes\_\_( ) 메서드에서 두 번 사용되는데, 우리는 단지 self.typecode로 그 값을 읽었다. Vector2d 객체가 그들 자신의 typecode 속성을 가지고 생성된 것이 아니므로 self.typecode는 기본적으로 Vector2d.typecode 클래스 속성을 가져온다.

그러나 존재하지 않는 객체 속성에 값을 저장하면, 새로운 객체 속성(에들 들면 typecode)을 생성하고 동일한 이름의 클래스 속성은 변경하지 않는다. 그 후부터는 객체가 self.typecode를 읽을 때 객체 자체의 typecode를 가져오므로, 동일한 이름의 클래스 속성을 가리게 된다. 그러면 각 객체가 서로 다른 typecode를 갖도록 커스터마이즈 할 수 있다.

In [31]:
""" [예제 9-13] 클래스에서 상속받은 typecode 속성을 설정해서 객체 커스터마이즈하기 """
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
print(dumpd)
print(len(dumpd)) # 기본적인 byte 표현은 17바이트 길이다.

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'
17


In [32]:
v1.typecode = 'f'
dumpf = bytes(v1)
print(dumpf)
print(len(dumpf)) # f로 변환

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'
9


In [33]:
print(Vector2d.typecode)
Vector2d.typecode = 'f'
print(Vector2d.typecode)

d
f


클래스 속성은 공개되어 있고 모든 서브클래스가 상속하므로, 클래스 데이터 속성을 커스터마이즈할 때는 클래스를 상속하는 것이 알반적인 방식이다.

In [34]:
class ShortVector2d(Vector2d):
    typecode = 'f'

sv = ShortVector2d(1/11, 1/27)
print('{!r}'.format(sv))
print(len(bytes(sv))) # 이전과 달리 17바이트가 아니라 9바이트다.

ShortVector2d(0.09090909090909091, 0.037037037037037035)
9
